In [7]:

#!pip install -U tensorflow==2.5.0
#!pip install numpy==1.19.2
#import tensorflow_hub as hub
#import tensorflow_text as text
#!pip install pyenchant
#!pip install transformers

import tensorflow as tf
import pandas as pd, numpy as np
import warnings, string
import spacy, re, nltk
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import enchant
import transformers
from transformers import AutoTokenizer, TFBertModel
from transformers.models.bert.modeling_flax_bert import BertConfig

#import keras
#from keras import initializers
#from keras.utils import io_utils
#from tensorflow.python.util.tf_export import keras_export
from tensorflow.keras.optimizers.legacy import Adam
#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, BinaryAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense

#from transformers import AutoTokenizer, AutoModelForMaskedLM

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
my_spacy= spacy.load('en_core_web_sm')

tweet = pd.read_csv('/content/drive/MyDrive/AAI_project_1/Covid-19 Tweet/Train 2.csv')
test_tweet_df = pd.read_csv('/content/drive/MyDrive/AAI_project_1/Covid-19 Tweet/Test 2.csv')
submission = pd.read_csv('/content/drive/MyDrive/AAI_project_1/Covid-19 Tweet/SampleSubmission 2.csv')

Y = tweet['target']



#column_features = ['ID', 'target']
#tweet = tweet.drop(column_features, axis = 1)



#

#REMOVE DIGITS
tweet['text'] = tweet['text'].str.replace(r'\d+', '')

#check for mispelled words and correct them
english_dict = enchant.Dict('en_US')

def correct_spelling(text):
    words = text.split()
    corrected_words = []
    for word in words:
        if not english_dict.check(word):
            suggestions = english_dict.suggest(word)
            if suggestions:
                corrected_words.append(suggestions[0])
            else:
                corrected_words.append(word)
        else:
            corrected_words.append(word)
    return ' '.join(corrected_words)

tweet['text'] = tweet['text'].apply(correct_spelling)
            
                

#TOKENIZE TWEETS
def tokenize(txt):
    tokens = re.split('\W+', txt)
    return tokens
           
tweet['text'] = tweet['text'].apply(lambda x: tokenize(x.lower()))
#print(tweet['text'])

#REMOVE STOPWORDS
my_spacy_stopwords = my_spacy.Defaults.stop_words

def remove_stopwords (tokenized_tweet):
    updated_tweet = [message for message in tokenized_tweet if message not in my_spacy_stopwords]
    return updated_tweet

tweet['text'] = tweet['text'].apply(lambda x: remove_stopwords(x))


                        
#Convert list of tokens back to strings
tweet['text'] = [' '.join(t) for t in tweet['text']]


#LEMMATIZATION
tweet['text'] = tweet['text'].apply(lambda x: " ".join([y.lemma_ for y in my_spacy(x)]))


X = tweet['text']

#model = AutoModelForMaskedLM.from_pretrained("vinai/bertweet-covid19-base-uncased")
#model = TFBertForMaskedLM.from_pretrained("vinai/bertweet-covid19-base-uncased")
Btokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')

bert = TFBertModel.from_pretrained('bert-large-uncased')
config = BertConfig()
config.vocab_size = Btokenizer.vocab_size
#Bmodel = TFBertModel.from_pretrained
#Btokenizer

#tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-covid19-base-uncased")
#Bmodel = TFBertModel.from_pretrained("vinai/bertweet-covid19-base-uncased")

#Convert text in dataframe into BERT input format
#print('max length of tweets', max([len(x.split()) for x in tweet.text]))
#tf.keras.utils.warmstart_embedding_matrix(
#    base_vocabulary, new_vocabulary, base_embeddings,
#    new_embeddings_initializer='uniform')

X_Train = Btokenizer(text = X.tolist(), add_special_tokens = True, max_length = 45,
                     truncation = True, padding = True, return_tensors = 'tf',
                     return_token_type_ids = False, return_attention_mask = True, verbose = True)

Y_train = tweet['target'].values

max_len = 45
input_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
input_mask = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

embeddings = bert(input_ids, attention_mask=input_mask)[1]
out =tf.keras.layers.Dropout(0.1)(embeddings)

out = Dense(128, activation = 'relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation = 'sigmoid')(out)

y = Dense(1, activation = 'sigmoid')(out)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

optimizer = Adam(learning_rate=6e-06, epsilon=1e-08, decay=0.01, clipnorm=1.0)

#Set loss and metrics
loss = BinaryCrossentropy(from_logits =  True)
metric = BinaryAccuracy('accuracy')

#compile the model
model.compile(optimizer = optimizer, loss = loss, metrics = metric)

#train the model
train_model = model.fit(x = {'input_ids': X_Train['input_ids'], 'attention_mask': X_Train['attention_mask']}, 
                        y = Y_train, epochs = 20, batch_size = 10)




Some layers from the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/20


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


529/529 [==============================] - 257s 394ms/step - loss: 0.5598 - accuracy: 0.7526
Epoch 2/20
529/529 [==============================] - 208s 393ms/step - loss: 0.3846 - accuracy: 0.8774
Epoch 3/20
529/529 [==============================] - 207s 392ms/step - loss: 0.3430 - accuracy: 0.8882
Epoch 4/20
529/529 [==============================] - 208s 393ms/step - loss: 0.3152 - accuracy: 0.9033
Epoch 5/20
529/529 [==============================] - 208s 393ms/step - loss: 0.3043 - accuracy: 0.9079
Epoch 6/20
529/529 [==============================] - 208s 393ms/step - loss: 0.2932 - accuracy: 0.9122
Epoch 7/20
529/529 [==============================] - 208s 393ms/step - loss: 0.2896 - accuracy: 0.9109
Epoch 8/20
529/529 [==============================] - 208s 393ms/step - loss: 0.2807 - accuracy: 0.9187
Epoch 9/20
529/529 [==============================] - 208s 392ms/step - loss: 0.2783 - accuracy: 0.9170
Epoch 10/20
529/529 [==============================] - 207s 392ms/step - lo

In [8]:

#REMOVE DIGITS
test_tweet_df['text'] = test_tweet_df['text'].str.replace(r'\d+', '')

#check for mispelled words and correct them
english_dict = enchant.Dict('en_US')

def correct_spelling(text):
    words = text.split()
    corrected_words = []
    for word in words:
        if not english_dict.check(word):
            suggestions = english_dict.suggest(word)
            if suggestions:
                corrected_words.append(suggestions[0])
            else:
                corrected_words.append(word)
        else:
            corrected_words.append(word)
    return ' '.join(corrected_words)

test_tweet_df['text'] = test_tweet_df['text'].apply(correct_spelling)
            
                

#TOKENIZE TWEETS
def tokenize(txt):
    tokens = re.split('\W+', txt)
    return tokens
           
test_tweet_df['text'] = test_tweet_df['text'].apply(lambda x: tokenize(x.lower()))
#print(tweet['text'])

#REMOVE STOPWORDS
my_spacy_stopwords = my_spacy.Defaults.stop_words

def remove_stopwords (tokenized_tweet):
    updated_tweet = [message for message in tokenized_tweet if message not in my_spacy_stopwords]
    return updated_tweet

test_tweet_df['text'] = test_tweet_df['text'].apply(lambda x: remove_stopwords(x))


                        
#Convert list of tokens back to strings
test_tweet_df['text'] = [' '.join(t) for t in test_tweet_df['text']]


#LEMMATIZATION
test_tweet_df['text'] = test_tweet_df['text'].apply(lambda x: " ".join([y.lemma_ for y in my_spacy(x)]))


X_new = test_tweet_df['text']

#Predict values from new dataset
X_Test = Btokenizer(text = X_new .tolist(), add_special_tokens = True, max_length = 45,
                     truncation = True, padding = True, return_tensors = 'tf',
                     return_token_type_ids = False, return_attention_mask = True, verbose = True)

predicted = model.predict( {'input_ids': X_Test['input_ids'], 'attention_mask': X_Test['attention_mask']})
#submission['ID'] = test_tweet_df.id
submission['target'] = predicted
submission.to_csv('/content/drive/MyDrive/AAI_project_1/Covid-19 Tweet/TweetSubmit.csv', index = False)
submission.to_csv('Submit.csv', index = False)
print('Done')

62/62 [==============================] - 26s 330ms/step
Done
